In [82]:
import pandas as pd 
import numpy as np 
import re
from openai import OpenAI
import ast
from tqdm import tqdm 
import matplotlib.pyplot as plt 
import plotly.express as px
import statsmodels.formula.api as smf
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr, pearsonr
import pickle
import itertools
import os
import tiktoken
from IPython.display import display, Markdown, Latex
from pandas import option_context


encoding = tiktoken.encoding_for_model("gpt-4-1106-preview")
client = OpenAI(api_key=open("/Users/mosobay/Dropbox (MIT)/research/openai_api", "r").read().strip())

In [25]:
len(encoding.encode("\n\n".join(df["rationale"].sample(800, random_state=2024))))

107323

In [24]:
df = pd.read_csv("./data/pairwise_comparison_rationales.csv")

In [91]:
for x in df["rationale"].sample(10):
    print(x+"\n\n")

Task B is more similar to the reference task in terms of team dynamics because both tasks involve analysis, evaluation, and decision-making based on specific criteria or information. In the reference task, participants must evaluate companies using investment criteria and come to a consensus on ranking them. Similarly, in Task B, participants must use their knowledge and reasoning to estimate quantities and strive for accuracy. Both tasks require critical thinking, sharing of knowledge, and collaborative decision-making to achieve the best possible outcome. Task A, on the other hand, is more about visual search and memory recall, which involves different cognitive processes and less analytical discussion compared to the reference task.


Task B is more similar to the reference task in terms of team dynamics because both tasks involve strategic planning, decision-making based on provided information, and a focus on a creative and economic outcome (writing an advertisement to generate cl

In [31]:
system_prompt = """You will be presented with the rationale behind comparisons generated from an exercise where an agent was asked to compare two different tasks in terms of how they related to a third task with respect to the team dynamics involved.
Each rationale is from a different comparison with different tasks, but the arguments made are useful in understanding the general space of tasks.
Use the logic presented to define a set of dimensions that are orthogonal to each other and collectively describe the differences between tasks.
For each dimension, present the dimension's name, description, and how it should be operationalized. 
Only present the dimensions in a list of JSON objects: [{"DIMENSION NAME", "DIMENSION DESCRIPTION", "DIMENSION OPERATIONALIZATION"}]"""

dimension_responses = []

for i in tqdm(range(10)):
    response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": "\n\n".join(df["rationale"].sample(800, random_state=i))}],
            temperature=0)
    dimension_responses.append(response)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [24:40<00:00, 148.06s/it]


In [72]:
df_dimensions = pd.concat([pd.DataFrame(ast.literal_eval(x.choices[0].message.content)).assign(iteration = index) for index, x in enumerate(dimension_responses)])
df_dimensions.loc[df_dimensions["iteration"] == 4, ["Dimension Name","Dimension Description","Dimension Operationalization"]] = df_dimensions.loc[df_dimensions["iteration"] == 4, ["DIMENSION NAME","DIMENSION DESCRIPTION","DIMENSION OPERATIONALIZATION"]].to_numpy()
df_dimensions = df_dimensions.drop(columns=["DIMENSION NAME","DIMENSION DESCRIPTION","DIMENSION OPERATIONALIZATION"])
df_dimensions.to_csv("./data/dimensionalities_from_pairwise.csv", index=False)

In [83]:
for iteration in range(10):
    with option_context('display.max_colwidth', None):
        display(df_dimensions.query("iteration == @iteration")[["Dimension Name", "Dimension Description"]])
    print("\n")

,Dimension Name,Dimension Description
0,Collaborative Decision-Making,"The extent to which a task requires participants to work together to analyze information, discuss differing opinions, and reach a consensus or agreement."
1,Role Differentiation,"The degree to which a task involves distinct roles or responsibilities for each team member, with each role contributing specific skills or knowledge to the team's objective."
2,Creative Synthesis,"The requirement for participants to generate new ideas or solutions by combining individual contributions into a single, cohesive output."
3,Strategic Interaction,"The level of strategic thinking and prediction required to consider the actions and potential reactions of others, both within the team and with external parties."
4,Information Sharing,The importance of communication and the exchange of information among team members to achieve a shared understanding and goal.
5,Real-Time Coordination,"The necessity for team members to interact and make decisions in a dynamic, real-time environment."
6,Individual vs. Collective Focus,The balance between tasks that prioritize individual performance and decision-making versus those that emphasize collective outcomes and team success.
7,Abstract Problem-Solving,The requirement for participants to engage in abstract thinking and apply non-concrete criteria to solve problems or generate outputs.
8,Time Constraint,"The presence of a fixed time limit within which participants must complete the task, adding pressure to the decision-making process."
9,Feedback Loop,"The presence of immediate or delayed feedback on team decisions and actions, which informs and influences subsequent team strategy."


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires team members to work together, share information, and make decisions collaboratively to achieve a common goal."
1,Role Differentiation,"The degree to which a task involves distinct roles or responsibilities for each team member, requiring role-specific skills and coordination."
2,Strategic Complexity,"The level of strategic thinking, planning, and adaptation required to achieve the task's objectives within a set of rules or constraints."
3,Creative Output,"The requirement for the task to produce a creative or non-standard solution, often involving idea generation and abstract thinking."
4,Consensus Requirement,"The necessity for team members to reach an agreement on a collective decision or output, often involving negotiation and synthesis of different viewpoints."
5,Real-Time Interaction,"The extent to which a task involves immediate, dynamic interaction among team members within a shared environment."
6,Competitive Element,"The presence of a competitive dynamic within the task, where individual or team performance is measured against others."
7,Information Asymmetry,"The distribution of information among team members, where not all participants have access to the same information, requiring sharing and integration to complete the task."
8,Objective Measurement,The ability to quantify the success of the task based on objective criteria or a clear scoring system.
9,Individual Autonomy,The level of independence granted to each team member to make decisions or contributions without the need for group consensus or collaboration.


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires team members to work together, share information, and make decisions collaboratively."
1,Role Differentiation,The degree to which a task involves distinct roles or responsibilities for each team member.
2,Strategic Complexity,"The level of strategic thinking, planning, and problem-solving required to achieve the task's goals."
3,Creative Requirement,"The necessity for creative input, idea generation, and innovative thinking within the task."
4,Consensus Building,The importance of reaching an agreement or unified decision among team members.
5,Information Asymmetry,The extent to which team members have access to different pieces of information that must be shared to complete the task.
6,Time Constraint,The presence of a time limit within which the team must complete the task.
7,Competitive Element,"The degree to which the task involves competition, either within the team or against an external opponent."
8,Feedback Loop,The presence of immediate or ongoing feedback that informs the team's decision-making process.
9,Individual vs. Collective Focus,The balance between tasks that require individual performance and those that emphasize collective team output.


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires team members to work together, share information, and make decisions collaboratively to achieve a common goal."
1,Role Differentiation,The degree to which a task involves distinct roles with specific responsibilities and functions within a team.
2,Strategic Complexity,"The level of strategic thinking, planning, and problem-solving required to achieve the objectives of a task."
3,Creative Requirement,"The necessity for original thought, idea generation, and creativity to produce an output or solve a problem."
4,Consensus Building,The importance of reaching agreement among team members through negotiation and discussion to produce a collective decision or output.
5,Time Constraint,"The presence of a time limit within which team members must complete the task, adding pressure to the decision-making process."
6,Information Asymmetry,The extent to which team members have access to different pieces of information that must be integrated to solve a task.
7,Objective Measurement,"The degree to which a task has a clear, quantifiable outcome or performance metric."
8,Competitive Element,"The presence of competition within the task, either between team members or against an external opponent."
9,Abstract Thinking,"The requirement for conceptual reasoning, ideation, and the application of abstract concepts to complete a task."


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires participants to work together, share information, and make decisions collaboratively."
1,Role Differentiation,The degree to which a task involves distinct roles or responsibilities for each team member.
2,Strategic Complexity,"The level of strategic thinking, planning, and problem-solving required to complete a task."
3,Creative Requirement,"The necessity for original thought, idea generation, and creativity in producing the task's outcome."
4,Consensus Building,"The need for team members to negotiate, discuss, and reach an agreement on a collective outcome."
5,Information Asymmetry,The extent to which team members have access to different pieces of information that must be combined to solve a task.
6,Time Constraint,The presence of a fixed time limit within which the task must be completed.
7,Objective Measurement,The degree to which the task's outcome can be objectively measured or has a clear right or wrong answer.
8,Spatial Coordination,The requirement for team members to navigate and manipulate elements within a shared visual or spatial environment.
9,Competitive Element,"The presence of competition within the task, either against an opponent or among team members."


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires team members to work together, share information, and make decisions collaboratively to achieve a common goal."
1,Role Differentiation,"The degree to which a task involves distinct, specialized roles or responsibilities for each team member."
2,Strategic Complexity,"The level of strategic thinking, planning, and problem-solving required to achieve the task's objectives within a set of rules or constraints."
3,Creative Requirement,"The necessity for original thought, ideation, and the generation of unique or non-redundant outputs as part of the task."
4,Information Symmetry,The extent to which team members have access to the same information or whether the task involves hidden or unshared information that must be communicated.
5,Objective Clarity,"The degree to which the task has a clear, measurable goal or outcome that can be objectively evaluated for success."
6,Time Constraint,"The presence of a fixed time limit within which the task must be completed, adding pressure and urgency to the team's efforts."
7,Competitive Element,"The extent to which the task involves competition, either within the team or against an external opponent, influencing individual and group strategies."
8,Spatial Coordination,The requirement for team members to navigate and interact with a shared physical or virtual space as part of the task.
9,Consensus Requirement,The necessity for team members to reach an agreement or unified decision as part of the task completion process.


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires participants to work together, share information, and make decisions as a group."
1,Role Differentiation,The degree to which a task involves distinct roles with specific responsibilities and required coordination among team members.
2,Strategic Complexity,"The level of strategic thinking, planning, and adaptation required to achieve the task's objectives."
3,Creative Output,The requirement for the team to generate creative ideas or solutions within a set framework.
4,Consensus Building,"The necessity for team members to negotiate, share perspectives, and reach an agreement on a collective decision."
5,Competitive Element,"The presence of a competitive dynamic within the team or against an external opponent, affecting individual and group outcomes."
6,Real-Time Interaction,The extent to which a task requires immediate decision-making and coordination among team members within a shared environment.
7,Information Sharing,The degree to which a task involves the distribution and synthesis of information among team members to complete the task.
8,Time Constraint,"The presence of a fixed time limit within which the team must complete the task, influencing strategy and urgency."
9,Objective Measurement,The extent to which the task outcome is based on objective criteria or quantifiable measures of success.


,Dimension Name,Dimension Description
0,Collaborative Decision-Making,"The extent to which a task requires team members to work together, share information, and reach a consensus on a collective outcome."
1,Role Differentiation,"The degree to which a task involves distinct roles or responsibilities for each team member, requiring specialized knowledge or skills."
2,Creative Problem-Solving,The need for team members to engage in creative thinking and generate multiple unique solutions or ideas within a set framework.
3,Strategic Planning,"The requirement for team members to analyze situations, predict outcomes, and develop a shared strategy to achieve a goal within a set of rules or constraints."
4,Real-Time Coordination,"The necessity for team members to interact and make decisions in a dynamic, real-time environment."
5,Information Integration,The extent to which a task requires team members to combine different pieces of information to solve a problem or reach a decision.
6,Abstract Thinking,The degree to which a task requires team members to use abstract reasoning and conceptualization to generate ideas or solutions.
7,Consensus-Building,"The need for team members to negotiate, discuss, and agree on a collective outcome that reflects the group's shared understanding or goal."
8,Competitive Dynamics,"The presence of individual goals that may conflict with those of other team members, leading to competitive rather than cooperative interactions."
9,Creative Synthesis,The requirement for team members to integrate individual ideas or inputs into a cohesive and creative output.


,Dimension Name,Dimension Description
0,Collaborative Decision-Making,"The extent to which a task requires team members to discuss, negotiate, and reach a consensus on a strategy or output."
1,Role Differentiation,The degree to which a task involves distinct roles with specific responsibilities and required coordination among team members.
2,Creative Output,The requirement for the team to generate ideas or solutions that are not predetermined and involve creativity or abstract thinking.
3,Strategic Planning,"The necessity for the team to analyze situations, predict outcomes, and develop a plan of action based on the rules or constraints of the task."
4,Information Sharing,The need for team members to share individual knowledge or data to collectively solve a problem or reach a decision.
5,Real-Time Interaction,"The involvement of direct, simultaneous actions or decisions by team members within a shared environment."
6,Objective Measurement,"The presence of a clear, quantifiable goal or performance metric that the team aims to achieve or optimize."
7,Individual vs. Collective Focus,The balance between tasks that emphasize individual contributions and those that prioritize collective team outcomes.
8,Feedback and Iteration,The extent to which a task provides feedback on team decisions and allows for iterative improvement based on that feedback.
9,Time Constraint,The presence of a fixed time limit within which the team must complete the task or make decisions.


,Dimension Name,Dimension Description
0,Collaboration Intensity,"The extent to which a task requires participants to work together, share information, and make decisions collaboratively to achieve a common goal."
1,Strategic Decision-Making,"The level of planning, anticipation of outcomes, and strategic thinking required to perform the task successfully."
2,Role Differentiation,"The degree to which a task involves distinct roles or responsibilities for each team member, contributing to the overall objective."
3,Creative Problem-Solving,"The requirement for generating unique, non-redundant ideas or solutions within a given framework."
4,Consensus Building,"The need for team members to discuss, negotiate, and reach an agreement on a collective decision or output."
5,Information Processing,"The extent to which a task requires the analysis, synthesis, and application of information to make decisions or produce an output."
6,Time Constraint,"The presence of a fixed time limit within which the task must be completed, adding pressure to the decision-making process."
7,Competitive vs. Cooperative Dynamics,"The nature of the interaction among participants, whether it is competitive with individual goals or cooperative with a shared objective."
8,Feedback and Adaptation,The task's provision of immediate feedback on performance and the requirement for participants to adapt their strategies based on that feedback.
9,Visual-Spatial Coordination,The requirement for participants to interact with and manipulate elements within a visual or spatial context to achieve the task's objectives.


In [88]:
with option_context('display.max_colwidth', None):
    display(df_dimensions.drop_duplicates(["Dimension Name"]).sort_values("Dimension Name"))

,Dimension Name,Dimension Description,Dimension Operationalization,iteration
7,Abstract Problem-Solving,The requirement for participants to engage in abstract thinking and apply non-concrete criteria to solve problems or generate outputs.,"Evaluate the level of abstract thinking required, the presence of non-concrete criteria or rules, and the degree of open-endedness in the task's solutions.",0
9,Abstract Thinking,"The requirement for conceptual reasoning, ideation, and the application of abstract concepts to complete a task.","Determined by the need for participants to think beyond standard classifications, generate ideas that are not immediately apparent, and solve problems that do not have a physical or action-oriented component.",3
0,Collaboration Intensity,"The extent to which a task requires team members to work together, share information, and make decisions collaboratively to achieve a common goal.","Measured by the number of interactions among team members, the necessity for consensus building, and the reliance on shared information to complete the task.",1
0,Collaborative Decision-Making,"The extent to which a task requires participants to work together to analyze information, discuss differing opinions, and reach a consensus or agreement.","Measure the number of decisions that require group agreement, the frequency and duration of team discussions, and the presence of mechanisms for resolving disagreements.",0
8,Competitive Dynamics,"The presence of individual goals that may conflict with those of other team members, leading to competitive rather than cooperative interactions.","Identify individual objectives within the team, assess the potential for conflict or competition, and evaluate the impact of individual actions on the collective outcome.",7
6,Competitive Element,"The presence of a competitive dynamic within the task, where individual or team performance is measured against others.","Evaluated based on the task's structure as a game or competition, the presence of an opponent or opposing team, and the scoring system that ranks performance.",1
7,Competitive vs. Cooperative Dynamics,"The nature of the interaction among participants, whether it is competitive with individual goals or cooperative with a shared objective.","Measured by the presence of individual versus group incentives, the potential for conflict or collaboration among team members, and the overall goal of the task (personal gain vs. team success).",9
4,Consensus Building,The importance of reaching an agreement or unified decision among team members.,"Determined by the requirement for group discussion, negotiation, and the integration of diverse opinions to produce a collective outcome.",2
4,Consensus Requirement,"The necessity for team members to reach an agreement on a collective decision or output, often involving negotiation and synthesis of different viewpoints.",Determined by the need for unanimous or majority agreement among team members and the process of integrating diverse opinions to reach a shared conclusion.,1
7,Consensus-Building,"The need for team members to negotiate, discuss, and agree on a collective outcome that reflects the group's shared understanding or goal.","Measure the frequency and importance of group agreement, the process of reconciling differing opinions, and the emphasis on a unified team decision.",7


In [89]:
df_dimensions.drop_duplicates(["Dimension Name"]).sort_values("Dimension Name").to_clipboard()